# Práctica Bloque 2 - IA

Autores: Sergi Mayol Matos, Alejandro Rodríguez Arguimbau <br>
Correos: sergi.mayol1@estudiant.uib.cat, alejandro.rodriguez7@estudiant.uib.cat <br>
Fecha: 06/12/2022

## Prerrequisitos

Para los siguientes apartados ([parte 1](#parte-1-preparación-de-los-datos) y [parte 2]()) se emplearán las siguientes librerías:

* [numpy](https://numpy.org/)
* [pandas](https://pandas.pydata.org/)
* [matplotlib](https://matplotlib.org/)
* [sklearn](https://scikit-learn.org/stable/)

> Nota: Para ver más información sobre las librerías, mirar el fichero [Pipfile](./Pipfile).

### Instalación de librerías

Para instalar las librerías, se debe ejecutar el siguiente comando:

```bash
pipenv install -d
```

> Nota: Se necesita tener instalado [pipenv](https://pypi.org/project/pipenv/).

### Importación de librerías

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Parte 1: Entrenamiento de modelos y comparación de resultados 

En la primera parte de la práctica se entrenarán diferentes modelos de clasificación (Regrsión lineal, Perceptrón, Random Forest) y se compararán sus resultados.

### Carga de datos

In [ ]:
df = pd.read_csv("./datos/dades.csv")
df.head()

### Análisis y procesamiento de datos

En esta sección se analizarán los datos de entrada para ver si hay que eliminar o modificar algún dato.

Para analizar los datos, comprobaremos la cantidad de datos que faltan en cada columna.

In [ ]:
def check_is_nan(df: pd.DataFrame):
    results = []
    col_names = []
    for col in df.columns:
        col_names.append(col)
        results.append(df[col].isna().sum())

    return {"col_names": col_names, "results": results, "percent": [x/len(df) for x in results]}


results = check_is_nan(df)

for i in range(len(results["col_names"])):
    print(
        f"{results['col_names'][i]}: {results['results'][i]} ({results['percent'][i]})")

In [ ]:
plt.bar(results["col_names"], results["percent"])
plt.title("Porcentaje de valores nulos por columna")
plt.xlabel("Columnas")
plt.ylabel("Porcentaje")
plt.xticks(rotation=90)
plt.show()

Se puede observar que existen 3 columnas con datos inexistentes:

* `Age`
* `Cabin`
* `Embarked`

Aunque de las 3 columnas, la columna con más datos faltantes, con diferencia, es `Cabin`, casi un 80%. Por ello, se eliminará esta columna.

Por lo que los datos que quedan, de momento, son:

In [ ]:
df.drop(['Cabin'], axis=1, inplace=True)
df.head()

Para las columnas `Age` y `Embarked`, se rellenarán los datos faltantes con la media y la moda, respectivamente. Se realizará la media de `Age`, ya que faltan aproximadamente un **20%** de los datos, y la moda de `Embarked`, ya que faltan muy pocos datos y no son datos numéricos.

In [ ]:
media_edad = df["Age"].mean()
df["Age"].fillna(media_edad, inplace=True)

moda_embarque = df["Embarked"].mode()[0]
df["Embarked"].fillna(moda_embarque, inplace=True)

df.head()

En este caso las columnas `Name` y `Ticket` no las vamos a utilizar y por eso las eliminamos del conjunto de datos.

In [ ]:
#df.drop(['Name'], axis=1, inplace=True)
#df.drop(['Ticket'], axis=1, inplace=True)
#df.head()

Convertimos la columna `Sex` en 0 y 1:
* Hombres: 0
* Mujeres: 1

In [ ]:
df['Sex']=df['Sex'].replace('male', 0)
df['Sex']=df['Sex'].replace('female', 1)
#clb = df.pop("Sex")
#ohe_clb = pd.get_dummies(clb, prefix='Sex')
#df = pd.concat([df.reset_index(drop=True), ohe_clb.reset_index(drop=True)], axis=1, sort=False)
df.head()


Obtenemos el tamaño de cada familia y la añadimos al conjunto de datos.

In [ ]:
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
# One-hot encoding para la columna Embarked
clb = df.pop("Embarked")
ohe_clb = pd.get_dummies(clb, prefix='Embarked')
df = pd.concat([df.reset_index(drop=True), ohe_clb.reset_index(drop=True)], axis=1, sort=False)
df.head()

Se puede dividir la columna `Age` en 5 grupos:
* 0-12  $\rightarrow$ Niños
* 13-17 $\rightarrow$ Adolescentes
* 18-45 $\rightarrow$ Adultos
* 46-64 $\rightarrow$ Adultos mayores
* 65+   $\rightarrow$ Ancianos

Además, se puede dividir la columna `Fare` en 4 grupos:
* 0-7.91   $\rightarrow$ Bajo
* 7.91-14.45 $\rightarrow$ Medio
* 14.45-31 $\rightarrow$ Alto
* 31+      $\rightarrow$ Muy alto

In [ ]:
# get max value of the column 'Fare'
max_value = df['Fare'].max()
# get max value of the column 'Age'
max_value_age = df['Age'].max()

print(max_value)
print(max_value_age)

for age in df:
    df['Age_type'] = pd.cut(df['Age'], bins=[0, 13, 18, 45, 65, 120], labels=['Child', 'Teenager', 'Adult', 'Senior', 'Elderly'])

for fare in df:
    df['Fare_type'] = pd.cut(df['Fare'], bins=[0, 7.91, 14.454, 31, 120], labels=['Low_fare', 'median_fare', 'Average_fare', 'high_fare'])

df.drop(['Age'], axis=1, inplace=True)
df.drop(['Fare'], axis=1, inplace=True)

df.head()

In [ ]:
# One-hot encoding para la columna Age_type 
clb = df.pop("Age_type")
ohe_clb = pd.get_dummies(clb, prefix='Age')
df = pd.concat([df.reset_index(drop=True), ohe_clb.reset_index(drop=True)], axis=1, sort=False)

# One-hot encoding para la columna Fare_type
clb = df.pop("Fare_type")
ohe_clb = pd.get_dummies(clb, prefix='Fare')
df = pd.concat([df.reset_index(drop=True), ohe_clb.reset_index(drop=True)], axis=1, sort=False)

df.head()

Se realiza la matriz de correlación

In [ ]:
corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')


Se observa que las columnas `Sex` y `Survived` tienen una correlación de 0.54, por lo que se puede decir que hay una relación entre la supervivencia y el sexo. También, se observa que `Parch` y `SibSp` ...

In [ ]:
#df.drop(['SibSp'], axis=1, inplace=True)
#df.drop(['Parch'], axis=1, inplace=True)
df.drop(['PassengerId'], axis=1, inplace=True)
df.drop(['Name'], axis=1, inplace=True)
df.drop(['Ticket'], axis=1, inplace=True)
# Las columnas de embarque no aportan nada a la predicción, por lo que las eliminamos
#df.drop(['Embarked_C'], axis=1, inplace=True)
#df.drop(['Embarked_Q'], axis=1, inplace=True)
#df.drop(['Embarked_S'], axis=1, inplace=True)

corr = df.corr()
corr.style.background_gradient(cmap='coolwarm')


In [ ]:
g = sns.pairplot(data=df, hue='Survived', palette = 'seismic',
                 size=1.2,diag_kind = 'kde',diag_kws=dict(shade=True),plot_kws=dict(s=10) )
g.set(xticklabels=[])

### Entrenamiento de modelos

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             confusion_matrix, mean_absolute_error, mean_squared_error,
                             classification_report)

training_ds = df.drop(['Survived'], axis=1)
target_ds = df['Survived']

# Dividimos el dataset en train y test
X_train, X_test, y_train, y_test = train_test_split(
    training_ds, target_ds, test_size=0.33, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape


#### Regresión logística

In [ ]:
from sklearn.linear_model import LogisticRegression

logisticRegr = LogisticRegression()

logisticRegr.fit(X_train, y_train)

log_y_pred = logisticRegr.predict(X_test)

print("Classification report: \n", classification_report(y_test, log_y_pred))

# Matriz de confusión
sns.heatmap(confusion_matrix(y_test, log_y_pred),
            annot=True, fmt="3.0f", cmap='coolwarm')


#### Perceptrón

In [ ]:
from sklearn.linear_model import Perceptron

perceptron = Perceptron(tol=1e-3, random_state=42)

perceptron.fit(X_train, y_train)

#x_intercept = -perceptron.intercept_[0] / perceptron.coef_[0][0]
x_intercept = (0, -perceptron.intercept_[0] / perceptron.coef_[0][0])
y_intercept = (-perceptron.intercept_[0] / perceptron.coef_[0][0], 0)

plt.plot(x_intercept, y_intercept)

In [ ]:
percep_y_pred = perceptron.predict(X_test)

print("Classification report: \n", classification_report(y_test, percep_y_pred))

# Plot the data, the prediction and the dividing line
# plt.scatter(X_test['Sex'], y_test, c=pred, cmap='coolwarm')

# plt.plot(x_intercept, y_intercept)
sns.heatmap(confusion_matrix(y_test, percep_y_pred),
            annot=True, fmt="3.0f", cmap='coolwarm')


#### Árbol de decisión - Random Forest

Primero hay que determinar los mejores párametros para el modelo. Para ello, se empleará la función `GridSearchCV` de la librería `sklearn`.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Create the parameter grid based on the results of random search
param_grid = {
    'bootstrap': [True, False],
    'max_depth': [80, 90, 100, 110],
    'min_samples_leaf': [1, 2, 3, 4],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 500, 700, 1000],
}

# Create a based model
rf = RandomForestClassifier()

# Instantiate the grid search model
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid,
                           cv=3, n_jobs=-1, verbose=2)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

result = grid_search.best_params_

print(result)


Una vez se han obtenido los mejores parámetros, se entrena el modelo con estos parámetros. Y se obtiene la precisión del modelo.

In [ ]:
rf = RandomForestClassifier(bootstrap=result['bootstrap'], max_depth=result['max_depth'],
                            min_samples_leaf=result['min_samples_leaf'], min_samples_split=result['min_samples_split'],
                            n_estimators=result['n_estimators'])

rf.fit(X_train, y_train)

rf_y_pred = rf.predict(X_test)

print("Classification report: \n", classification_report(y_test, rf_y_pred))


Y se obtiene la matriz de confusión.

In [ ]:
# Matriz de confusión
sns.heatmap(confusion_matrix(y_test, rf_y_pred),
            annot=True, fmt="3.0f", cmap='coolwarm')

### Evaluación y comparción de resultados

In [ ]:
modelos = pd.DataFrame({
    'Modelo': ['Regresión Logística', 'Perceptrón', 'Random Forest'],
    'Score': [round(accuracy_score(y_test, log_y_pred), 2), round(accuracy_score(y_test, percep_y_pred), 2), round(accuracy_score(y_test, rf_y_pred), 2)],
    'Precision': [round(precision_score(y_test, log_y_pred), 2), round(precision_score(y_test, percep_y_pred), 2), round(precision_score(y_test, rf_y_pred), 2)],
    'Recall': [round(recall_score(y_test, log_y_pred), 2), round(recall_score(y_test, percep_y_pred), 2), round(recall_score(y_test, rf_y_pred), 2)],
    'F1': [round(f1_score(y_test, log_y_pred), 2), round(f1_score(y_test, percep_y_pred), 2), round(f1_score(y_test, rf_y_pred), 2)],
    'MAE': [round(mean_absolute_error(y_test, log_y_pred), 2), round(mean_absolute_error(y_test, percep_y_pred), 2), round(mean_absolute_error(y_test, rf_y_pred), 2)],
    'MSE': [round(mean_squared_error(y_test, log_y_pred), 2), round(mean_squared_error(y_test, percep_y_pred), 2), round(mean_squared_error(y_test, rf_y_pred), 2)]
})

modelos.sort_values(by='Score', ascending=False)

## Parte 2: Importancia y selección de características

El modelo se basará en características como el nombre de los pasajeros, el sexo o la edad. Para la realización de los modelos se ha hecho una selección de estas características.

La importancia de una característica viene determinada por el valor que esta proporciona al conjunto de datos. 

Por ejemplo: las columnas Name y Ticket no nos proporcionan ningún valor útil al conjunto de datos y por ello, se han eliminado y no se han tenido en cuenta. 

En este apartado, se analizará la importancia de cada una de las características para cada modelo de entrenamiento.

- Feature importance Random Forest
- Feature importance Regresión Lineal
- Feature importance Perceptron
- https://machinelearningmastery.com/calculate-feature-importance-with-python/

In [ ]:
print("Feature importances for Random Forest:")
importance = rf.feature_importances_
for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))
plt.bar([x for x in range(len(importance))], importance)
plt.show()


In [ ]:
print("Feature importances for Logistic Regression:")
importance = logisticRegr.coef_[0]
for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))
plt.bar([x for x in range(len(importance))], importance)
plt.show()

In [ ]:
print("Feature importances for Perceptron:")
importance = perceptron.coef_[0]
for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))
plt.bar([x for x in range(len(importance))], importance)
plt.show()

- Selección de los mejores parámetros para cada modelo